# Chroma Indexing and RAG Examples

In [27]:
!pip install haystack-ai ollama-haystack
!pip install nltk
!pip install sentence-transformers
!pip install "transformers[torch]"

  Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached scipy-1.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached scipy-1.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.3 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)


In [3]:
import os
import random
import torch
from pathlib import Path
from pprint import pprint
from haystack import Pipeline
from haystack.dataclasses import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import (
    SentenceTransformersDocumentEmbedder,
    SentenceTransformersTextEmbedder,
)
from haystack.components.converters import TextFileToDocument
from haystack.components.writers import DocumentWriter
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import ComponentDevice

In [4]:
!ls -la ../data/parsed_files

total 25508
drwxrwxr-x 2 skitarii skitarii   12288 апр 16 18:36  .
drwxrwxr-x 8 skitarii skitarii    4096 апр 16 17:04  ..
-rw-r--r-- 1 root     root      166032 апр 16 18:09  04_Великие_музеи_мира_Египетский_музей_2011.md
-rw-r--r-- 1 root     root         113 апр 16 17:53  1d8d62aa-6d7a-4544-8378-51cffe22db3f.md
-rw-r--r-- 1 root     root         132 апр 16 17:53  63вгдеёжзиклмнуфхцчшщырврарвррырыррсрчррфрырыфрырвррветфддф_1.md
-rw-r--r-- 1 root     root         132 апр 16 17:53  63вгдеёжзиклмнуфхцчшщырврарвррырыррсрчррфрырыфрырвррветфддф.md
-rw-r--r-- 1 root     root         102 апр 16 17:53  64fsdgdfigidfiohgopdfpososdoidfsgsdfgsdfgdsfdsfgdfsdfgsdsdf1.md
-rw-r--r-- 1 root     root        3805 апр 16 17:53  authors_1.md
-rw-r--r-- 1 root     root         438 апр 16 17:53  authors1.md
-rw-r--r-- 1 root     root         780 апр 16 17:53 'authors (1).md'
-rw-r--r-- 1 root     root        2201 апр 16 17:53 'authors (2) (2)2315676354225745865453634524523452345234523452345234523452345.m

In [5]:
file_paths = ["../data/parsed_files" / Path(name) for name in os.listdir("../data/parsed_files")]

In [6]:
len(file_paths)

148

In [8]:
import os
from pathlib import Path
from haystack.dataclasses import Document

# Пути к вашим уже распаршенным файлам
# Здесь мы предполагаем, что это обычные текстовые файлы (.txt, .md и т.п.)
base_dir = Path("../data/parsed_files")
file_paths = [base_dir / name for name in os.listdir(base_dir) if (base_dir / name).is_file()]

raw_docs = []
for path in file_paths:
    # Читаем весь текст из файла
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    # Создаём Document, кладём в meta название и полный путь
    doc = Document(
        content=text,
        meta={
            "name": path.name,
            "path": str(path.resolve())
        }
    )
    raw_docs.append(doc)

# Проверим, сколько документов получилось
print(f"Loaded {len(raw_docs)} documents")
for doc in raw_docs[:3]:
    print("—", doc.meta["name"], "(", len(doc.content), "символов )")

Loaded 148 documents
— dkp-1.md ( 3942 символов )
— DKP_1.md ( 4804 символов )
— Инструкция работника для подачи заявления на социальные льготы.md ( 4861 символов )


In [10]:
# 3. Индексирование
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
indexing = Pipeline()
indexing.add_component("cleaner", DocumentCleaner())
indexing.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=2))
indexing.add_component(
    "doc_embedder",
    SentenceTransformersDocumentEmbedder(
        model="thenlper/gte-large",
        device=ComponentDevice.from_str("cuda:0"),
        meta_fields_to_embed=["title"]
    )
)
indexing.add_component("writer", DocumentWriter(document_store=document_store))
indexing.connect("cleaner", "splitter")
indexing.connect("splitter", "doc_embedder")
indexing.connect("doc_embedder", "writer")

# Запуск индексирования
indexing.run({"cleaner": {"documents": raw_docs}})

No abbreviations file found for en. Using default abbreviations.
Document ID 935666b5c14f2e7f627c0c14192892ca0621a76ba81c673ad67a5265a808a298 has an empty content. Skipping this document.


Batches:   0%|          | 0/1742 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder())
query_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")

query = "Who lives in Berlin?"

indexing_pipeline.run({"documents": documents})
result = query_pipeline.run({"text_embedder":{"text": query}})

print(result['retriever']['documents'][0])

In [ ]:
Как менялось предназначение здания музея Прадо на протяжении истории, и какие ключевые события повлияли на его становление?

In [23]:
# 4. Настройка генератора на Ollama
from haystack.utils import Secret

MODEL_NAME = "hf.co/IlyaGusev/saiga_yandexgpt_8b_gguf:Q6_K"
generator = OpenAIGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={
        "temperature": 0.8
    }
)

In [22]:
# quick check
import rich
rich.print(generator.run("Please write a rhyme about Italy."))

{
    'replies': [
        'В Италии солнце смеется,\nИ пицца там в каждом доме. Колибри\nЛетает меж старинных колоколов;\nВина 
сладкие, как мечты влюбленных.\n\nГде Рафаэль кисть своею рисует,\nА Верди музыку страстно дарит.\nИталия — страна,
где сердце тает,\nКогда душа с природой все сливает.'
    ],
    'meta': [
        {
            'model': 'hf.co/IlyaGusev/saiga_yandexgpt_8b_gguf:Q6_K',
            'index': 0,
            'finish_reason': 'stop',
            'usage': {
                'completion_tokens': 74,
                'prompt_tokens': 31,
                'total_tokens': 105,
                'completion_tokens_details': None,
                'prompt_tokens_details': None
            }
        }
    ]
}

In [ ]:
# 7. Утилита для получения ответа
def get_generative_answer(query: str):
    result = rag.run({
        "text_embedder": {"text": query},
        "prompt_builder": {"query": query}
    })
    answer = result["llm"]["replies"][0]
    print(f"Вопрос: {query}\nОтвет: {answer}\n")

# Пример
get_generative_answer("Какой стиль у группы The Cure?")
get_generative_answer("Была ли Земля плоской?")
